In [ ]:
import pandas as pd

from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
from transformers import TextClassificationPipeline
from transformers import DistilBertForSequenceClassification

import torch
import tensorflow as tf
import pandas as pd
import json
import gc

import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
import statsmodels.api as sm
import statsmodels.formula.api as smf
%matplotlib inline
from sklearn.preprocessing import LabelEncoder

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopw = stopwords.words('english')

import seaborn as sns
import matplotlib.pyplot as plt
from plotly.offline import iplot

from tqdm import tqdm
import mpl_toolkits.mplot3d
import numpy as np
from sklearn.cluster import KMeans

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


***Ao testar esta primeira versão, usar apenas atributos referentes ao tipo criatura. Generalizarei para as demais cartas nas próximas versões***

# Criando o dataframe, categorizando e dividindo as amostras por tipo de carta

In [ ]:
prices = pd.read_csv('/content/drive/Shareddrives/Ufla/Pattern_Recognition/cardPrices.csv')

prices


,cardFinish,currency,date,gameAvailability,price,priceProvider,providerListing,uuid
0,normal,USD,2024-02-03,mtgo,0.03,cardhoarder,retail,f182e364-0439-5594-a6e6-75f7889ccf45
1,normal,USD,2024-02-03,mtgo,0.18,cardhoarder,retail,330deaa3-dd7a-52a8-bfbc-b323cd16a409
2,normal,USD,2024-02-03,mtgo,0.02,cardhoarder,retail,79e36956-b91f-580f-8309-7d9585a67560
3,normal,USD,2024-02-03,mtgo,0.07,cardhoarder,retail,6afb2b4c-530a-57d5-8e7f-871239f6fa05
4,normal,USD,2024-02-03,mtgo,0.02,cardhoarder,retail,b1fc2762-92aa-5a14-8509-a59cb611e376
...,...,...,...,...,...,...,...,...
517004,normal,USD,2024-02-03,paper,0.15,cardkingdom,buylist,eb77951a-574b-502d-b985-1d206a261261
517005,normal,USD,2024-02-03,paper,0.22,cardkingdom,buylist,c61b08e0-477a-5afe-bea0-0886f216e97a
517006,normal,USD,2024-02-03,paper,0.36,cardkingdom,buylist,9756f4b9-c4d2-5d60-b1ab-656a74d7957a
517007,normal,USD,2024-02-03,paper,2124.98,cardsphere,retail,72e22db3-a046-52b2-bd16-b1bb470f16d5


In [ ]:
cards = pd.read_csv('/content/drive/Shareddrives/Ufla/Pattern_Recognition/cards.csv')

cards

<ipython-input-175-d1c4e8af6658>:1: DtypeWarning:

Columns (3,7,12,16,20,23,25,26,30,31,32,33,35,39,40,41,47,52,53,61,62,66,68) have mixed types. Specify dtype option on import or set low_memory=False.



,artist,artistIds,asciiName,attractionLights,availability,boosterTypes,borderColor,cardParts,colorIdentity,colorIndicator,...,subsets,subtypes,supertypes,text,toughness,type,types,uuid,variations,watermark
0,Pete Venters,d54c4a1a-c0c5-4834-84db-125d341f3ad8,NaN,NaN,"mtgo, paper",default,black,NaN,W,NaN,...,NaN,"Human, Cleric",NaN,First strike (This creature deals combat damag...,4,Creature — Human Cleric,Creature,5f8287b1-5bb6-5f4c-ad17-316a40d5bb0c,b7c19924-b4bf-56fc-aa73-f586e940bd42,NaN
1,Pete Venters,d54c4a1a-c0c5-4834-84db-125d341f3ad8,NaN,NaN,"mtgo, paper",default,black,NaN,W,NaN,...,NaN,"Human, Cleric",NaN,First strike (This creature deals combat damag...,4,Creature — Human Cleric,Creature,b7c19924-b4bf-56fc-aa73-f586e940bd42,5f8287b1-5bb6-5f4c-ad17-316a40d5bb0c,NaN
2,Volkan Baǵa,93bec3c0-0260-4d31-8064-5d01efb4153f,NaN,NaN,"mtgo, paper",default,black,NaN,W,NaN,...,NaN,Angel,NaN,Flying\nWhen Angel of Mercy enters the battlef...,3,Creature — Angel,Creature,57aaebc1-850c-503d-9f6e-bb8d00d8bf7c,8fd4e2eb-3eb4-50ea-856b-ef638fa47f8a,NaN
3,Volkan Baǵa,93bec3c0-0260-4d31-8064-5d01efb4153f,NaN,NaN,"mtgo, paper",default,black,NaN,W,NaN,...,NaN,Angel,NaN,Flying\nWhen Angel of Mercy enters the battlef...,3,Creature — Angel,Creature,8fd4e2eb-3eb4-50ea-856b-ef638fa47f8a,57aaebc1-850c-503d-9f6e-bb8d00d8bf7c,NaN
4,Mark Zug,48e2b98c-5467-4671-bd42-4c3746115117,NaN,NaN,"mtgo, paper",default,black,NaN,W,NaN,...,NaN,NaN,NaN,Target creature gets +3/+3 and gains flying un...,NaN,Sorcery,Sorcery,55bd38ca-dc73-5c06-8f80-a6ddd2f44382,c5655330-5131-5f40-9d3e-0549d88c6e9e,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92800,Campbell White,d281eab4-463a-4ba8-9039-8943737960a0,NaN,NaN,"mtgo, paper",NaN,black,NaN,U,NaN,...,NaN,NaN,NaN,Kicker {1}{U} (You may pay an additional {1}{U...,NaN,Instant,Instant,3f492516-7767-5ed7-a1d4-e3f7c06aee2f,dd604910-0e81-5d56-b022-e08f21de0879,planeswalker
92801,Jason Rainville,6ed7e669-579b-443d-b223-e5cbcb2a7483,NaN,NaN,"mtgo, paper",NaN,black,NaN,B,NaN,...,NaN,NaN,NaN,Kicker {2}{B} (You may pay an additional {2}{B...,NaN,Sorcery,Sorcery,3f9a0369-5fe7-5aee-85fe-3cfaacd275af,d947d9cd-f855-5496-b5de-88006b49865f,planeswalker
92802,Campbell White,d281eab4-463a-4ba8-9039-8943737960a0,NaN,NaN,"mtgo, paper",NaN,black,NaN,R,NaN,...,NaN,NaN,NaN,Kicker {5} (You may pay an additional {5} as y...,NaN,Sorcery,Sorcery,97577e9e-69a9-5a8b-9c24-a72703790046,dbd65728-bba8-536e-a908-5dfa56068dcb,planeswalker
92803,Jonas De Ro,561ebf9e-8d93-4b57-8156-8826d0c19601,NaN,NaN,"mtgo, paper",NaN,black,NaN,G,NaN,...,NaN,NaN,NaN,Sacrifice a land. Search your library for up t...,NaN,Instant,Instant,deb51cbd-b890-5b2d-9d6f-7b896e16c6fd,7ef6cffb-335a-5831-8319-20b6a020f1c4,planeswalker


In [ ]:
df = cards.merge(prices, left_on= 'uuid', right_on= 'uuid')

df

,artist,artistIds,asciiName,attractionLights,availability,boosterTypes,borderColor,cardParts,colorIdentity,colorIndicator,...,uuid,variations,watermark,cardFinish,currency,date,gameAvailability,price,priceProvider,providerListing
0,Pete Venters,d54c4a1a-c0c5-4834-84db-125d341f3ad8,NaN,NaN,"mtgo, paper",default,black,NaN,W,NaN,...,5f8287b1-5bb6-5f4c-ad17-316a40d5bb0c,b7c19924-b4bf-56fc-aa73-f586e940bd42,NaN,normal,USD,2024-02-03,mtgo,0.04,cardhoarder,retail
1,Pete Venters,d54c4a1a-c0c5-4834-84db-125d341f3ad8,NaN,NaN,"mtgo, paper",default,black,NaN,W,NaN,...,5f8287b1-5bb6-5f4c-ad17-316a40d5bb0c,b7c19924-b4bf-56fc-aa73-f586e940bd42,NaN,foil,USD,2024-02-03,mtgo,0.02,cardhoarder,retail
2,Pete Venters,d54c4a1a-c0c5-4834-84db-125d341f3ad8,NaN,NaN,"mtgo, paper",default,black,NaN,W,NaN,...,5f8287b1-5bb6-5f4c-ad17-316a40d5bb0c,b7c19924-b4bf-56fc-aa73-f586e940bd42,NaN,normal,USD,2024-02-03,paper,0.11,tcgplayer,retail
3,Pete Venters,d54c4a1a-c0c5-4834-84db-125d341f3ad8,NaN,NaN,"mtgo, paper",default,black,NaN,W,NaN,...,5f8287b1-5bb6-5f4c-ad17-316a40d5bb0c,b7c19924-b4bf-56fc-aa73-f586e940bd42,NaN,foil,USD,2024-02-03,paper,0.59,tcgplayer,retail
4,Pete Venters,d54c4a1a-c0c5-4834-84db-125d341f3ad8,NaN,NaN,"mtgo, paper",default,black,NaN,W,NaN,...,5f8287b1-5bb6-5f4c-ad17-316a40d5bb0c,b7c19924-b4bf-56fc-aa73-f586e940bd42,NaN,normal,EUR,2024-02-03,paper,0.05,cardmarket,retail
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528554,Colin Boyer,9c201dbe-db56-429a-87e6-189ea70c2632,NaN,NaN,"mtgo, paper",NaN,black,NaN,"R, W",NaN,...,46407d93-df48-5161-95fe-f24086746663,f3434acf-7796-572c-a889-487c84cf7948,planeswalker,foil,EUR,2024-02-03,paper,0.04,cardmarket,retail
528555,Colin Boyer,9c201dbe-db56-429a-87e6-189ea70c2632,NaN,NaN,"mtgo, paper",NaN,black,NaN,"R, W",NaN,...,46407d93-df48-5161-95fe-f24086746663,f3434acf-7796-572c-a889-487c84cf7948,planeswalker,normal,USD,2024-02-03,paper,0.10,cardkingdom,buylist
528556,Colin Boyer,9c201dbe-db56-429a-87e6-189ea70c2632,NaN,NaN,"mtgo, paper",NaN,black,NaN,"R, W",NaN,...,46407d93-df48-5161-95fe-f24086746663,f3434acf-7796-572c-a889-487c84cf7948,planeswalker,foil,USD,2024-02-03,paper,0.10,cardkingdom,buylist
528557,Colin Boyer,9c201dbe-db56-429a-87e6-189ea70c2632,NaN,NaN,"mtgo, paper",NaN,black,NaN,"R, W",NaN,...,46407d93-df48-5161-95fe-f24086746663,f3434acf-7796-572c-a889-487c84cf7948,planeswalker,foil,USD,2024-02-03,paper,0.49,cardkingdom,retail


In [ ]:
df.columns

Index(['artist', 'artistIds', 'asciiName', 'attractionLights', 'availability',
       'boosterTypes', 'borderColor', 'cardParts', 'colorIdentity',
       'colorIndicator', 'colors', 'defense', 'duelDeck', 'edhrecRank',
       'edhrecSaltiness', 'faceConvertedManaCost', 'faceFlavorName',
       'faceManaValue', 'faceName', 'finishes', 'flavorName', 'flavorText',
       'frameEffects', 'frameVersion', 'hand', 'hasAlternativeDeckLimit',
       'hasContentWarning', 'hasFoil', 'hasNonFoil', 'isAlternative',
       'isFullArt', 'isFunny', 'isOnlineOnly', 'isOversized', 'isPromo',
       'isRebalanced', 'isReprint', 'isReserved', 'isStarter',
       'isStorySpotlight', 'isTextless', 'isTimeshifted', 'keywords',
       'language', 'layout', 'leadershipSkills', 'life', 'loyalty', 'manaCost',
       'manaValue', 'name', 'number', 'originalPrintings',
       'originalReleaseDate', 'originalText', 'originalType', 'otherFaceIds',
       'power', 'printings', 'promoTypes', 'rarity', 'rebalancedPrint

In [ ]:
unwanted_list = ('artist', 'artistIds', 'asciiName', 'attractionLights', 'availability',
       'boosterTypes', 'borderColor', 'cardParts', 'colorIdentity',
       'colorIndicator', 'colors', 'defense', 'duelDeck', 'manaCost',
       'faceConvertedManaCost', 'faceFlavorName', 'edhrecRank',	'edhrecSaltiness',
       'faceManaValue', 'faceName', 'finishes', 'flavorName', 'flavorText',
       'frameEffects', 'frameVersion', 'hand', 'hasAlternativeDeckLimit',
       'hasContentWarning', 'hasFoil', 'hasNonFoil', 'isAlternative',
       'isFullArt', 'isFunny', 'isOnlineOnly', 'isOversized', 'isPromo',
       'isRebalanced', 'isReprint', 'isReserved', 'isStarter',
       'isStorySpotlight', 'isTextless', 'isTimeshifted', 'keywords',
       'language', 'layout', 'leadershipSkills', 'life',
       'number', 'originalPrintings',
       'originalReleaseDate', 'originalText', 'otherFaceIds',
       'printings', 'promoTypes', 'rebalancedPrintings',
       'relatedCards', 'securityStamp', 'setCode', 'side', 'signature',
       'sourceProducts', 'subsets', 'subtypes', 'supertypes', 'type', 'types', 'uuid', 'variations', 'watermark',
       'cardFinish', 'currency', 'date', 'gameAvailability',
       'priceProvider', 'providerListing')

In [ ]:
df_wanted = df.drop(columns=[col for col in df if col in unwanted_list])

df_wanted

,loyalty,manaValue,name,originalType,power,rarity,text,toughness,price
0,NaN,7.0,Ancestor's Chosen,Creature - Human Cleric,4,uncommon,First strike (This creature deals combat damag...,4,0.04
1,NaN,7.0,Ancestor's Chosen,Creature - Human Cleric,4,uncommon,First strike (This creature deals combat damag...,4,0.02
2,NaN,7.0,Ancestor's Chosen,Creature - Human Cleric,4,uncommon,First strike (This creature deals combat damag...,4,0.11
3,NaN,7.0,Ancestor's Chosen,Creature - Human Cleric,4,uncommon,First strike (This creature deals combat damag...,4,0.59
4,NaN,7.0,Ancestor's Chosen,Creature - Human Cleric,4,uncommon,First strike (This creature deals combat damag...,4,0.05
...,...,...,...,...,...,...,...,...,...
528554,NaN,3.0,Kargan Warleader,NaN,3,uncommon,Other Warriors you control get +1/+1.,3,0.04
528555,NaN,3.0,Kargan Warleader,NaN,3,uncommon,Other Warriors you control get +1/+1.,3,0.10
528556,NaN,3.0,Kargan Warleader,NaN,3,uncommon,Other Warriors you control get +1/+1.,3,0.10
528557,NaN,3.0,Kargan Warleader,NaN,3,uncommon,Other Warriors you control get +1/+1.,3,0.49


In [ ]:
df_wanted.drop_duplicates('name', inplace = True)

In [ ]:
df_wanted.dropna(subset='originalType',inplace=True)

In [ ]:
df_wanted.fillna(-1,inplace=True)

In [ ]:
df_wanted['raridade'] = df_wanted['rarity'].astype('category').cat.codes

In [ ]:
df_wanted['classified_text'] = df_wanted['price'].astype('category').cat.codes

In [ ]:
df_wanted

,loyalty,manaValue,name,originalType,power,rarity,text,toughness,price,raridade,classified_text
0,-1,7.0,Ancestor's Chosen,Creature - Human Cleric,4,uncommon,First strike (This creature deals combat damag...,4,0.04,4,3
14,-1,5.0,Angel of Mercy,Creature - Angel,3,uncommon,Flying\nWhen Angel of Mercy enters the battlef...,3,0.04,4,3
28,-1,3.0,Angelic Blessing,Sorcery,-1,common,Target creature gets +3/+3 and gains flying un...,-1,0.04,0,3
42,-1,5.0,Angelic Chorus,Enchantment,-1,rare,Whenever a creature enters the battlefield und...,-1,0.02,2,1
53,-1,2.0,Angelic Wall,Creature - Wall,0,common,Defender (This creature can't attack.)\nFlying,4,0.04,0,3
...,...,...,...,...,...,...,...,...,...,...,...
523560,-1,6.0,Eternity Vessel,Artifact,-1,mythic,Eternity Vessel enters the battlefield with X ...,-1,0.02,1,1
523600,-1,2.0,Hedron Scrabbler,Artifact Creature — Construct,1,common,Landfall — Whenever a land enters the battlefi...,1,0.03,0,2
523610,-1,4.0,Khalni Gem,Artifact,-1,uncommon,"When Khalni Gem enters the battlefield, return...",-1,0.03,4,2
523739,-1,0.0,"Magosi, the Waterveil",Land,-1,rare,"Magosi, the Waterveil enters the battlefield t...",-1,0.02,2,1


In [ ]:
df_wanted['classified_text'].min()

0

In [ ]:
df_name = df_wanted['name']
df_wanted.drop(columns=['name'],inplace=True)
df_wanted.set_index(df_name, inplace=True)

In [ ]:
df_wanted2 = df_wanted.drop(columns='loyalty')
notributes = ['power','toughness']
df_wanted3 = df_wanted2.drop(columns= [col for col in df_wanted2 if col in notributes])
df_wanted4 = df_wanted3.drop(columns='manaValue')
df_wanted5 = df_wanted.drop(columns= [col for col in df_wanted2 if col in notributes])

In [ ]:
df_art = df_wanted2.loc[df_wanted['originalType'].str.contains('Artifact')]
df_mag = df_wanted3.loc[df_wanted['originalType'].str.contains('Sorcery')]
df_inst = df_wanted3.loc[df_wanted['originalType'].str.contains('Instant')]
df_enc = df_wanted3.loc[df_wanted['originalType'].str.contains('Enchantment')]
df_terr = df_wanted4.loc[df_wanted['originalType'].str.contains('Land')]
df_crea = df_wanted2.loc[df_wanted['originalType'].str.contains('Creature')]
df_planes = df_wanted5.loc[df_wanted['originalType'].str.contains('Planeswalker')]

In [ ]:
df_art

,manaValue,originalType,power,rarity,text,toughness,price,raridade,classified_text
name,,,,,,,,,
Angel's Feather,2.0,Artifact,-1,uncommon,"Whenever a player casts a white spell, you may...",-1,0.04,4,3
Bottle Gnomes,3.0,Artifact Creature - Gnome,1,uncommon,Sacrifice Bottle Gnomes: You gain 3 life.,3,0.03,4,2
Chimeric Staff,4.0,Artifact,-1,rare,{X}: Chimeric Staff becomes an X/X Construct a...,-1,0.02,2,1
Chromatic Star,1.0,Artifact,-1,uncommon,"{1}, {T}, Sacrifice Chromatic Star: Add one ma...",-1,0.04,4,3
Citanul Flute,5.0,Artifact,-1,rare,"{X}, {T}: Search your library for a creature c...",-1,0.02,2,1
...,...,...,...,...,...,...,...,...,...
Sentinel Totem,1.0,Artifact,-1,uncommon,"When Sentinel Totem enters the battlefield, sc...",-1,0.03,4,2
Sleek Schooner,3.0,Artifact — Vehicle,4,uncommon,Crew 1 (Tap any number of creatures you contro...,3,0.03,4,2
Eternity Vessel,6.0,Artifact,-1,mythic,Eternity Vessel enters the battlefield with X ...,-1,0.02,1,1


In [ ]:
df_crea

,manaValue,originalType,power,rarity,text,toughness,price,raridade,classified_text
name,,,,,,,,,
Ancestor's Chosen,7.0,Creature - Human Cleric,4,uncommon,First strike (This creature deals combat damag...,4,0.04,4,3
Angel of Mercy,5.0,Creature - Angel,3,uncommon,Flying\nWhen Angel of Mercy enters the battlef...,3,0.04,4,3
Angelic Wall,2.0,Creature - Wall,0,common,Defender (This creature can't attack.)\nFlying,4,0.04,0,3
Aven Cloudchaser,4.0,Creature - Bird Soldier,2,common,Flying (This creature can't be blocked except ...,2,0.04,0,3
Ballista Squad,4.0,Creature - Human Rebel,2,uncommon,"{X}{W}, {T}: Ballista Squad deals X damage to ...",2,0.03,4,2
...,...,...,...,...,...,...,...,...,...
Timbermaw Larva,4.0,Creature — Beast,2,common,"Whenever Timbermaw Larva attacks, it gets +1/+...",2,0.03,0,2
Turntimber Ranger,5.0,Creature — Elf Scout Ally,2,rare,Whenever Turntimber Ranger or another Ally ent...,2,0.02,2,1
Zendikar Farguide,5.0,Creature — Elemental,3,common,Forestwalk (This creature can't be blocked as ...,3,0.04,0,3


In [ ]:
df_crea.loc[df_crea['classified_text'].idxmax()]

manaValue                                                        5.0
originalType                                       Creature - Legend
power                                                              2
rarity                                                          rare
text               Horsemanship (This creature can't be blocked e...
toughness                                                          3
price                                                          166.1
raridade                                                           2
classified_text                                                 1075
Name: Yuan Shao, the Indecisive, dtype: object

In [ ]:
df_crea['originalType'].values

array(['Creature - Human Cleric', 'Creature - Angel', 'Creature - Wall',
       ..., 'Creature — Elemental', 'Artifact Creature — Construct',
       'Creature — Sphinx Rogue'], dtype=object)

In [ ]:
to_drop2 = ['*','1+*','2+*','-1','+1','+3','3.5','?','∞','2.5','1.5','99','.5','*²','+4','+0','+2',-1]
for k in range(len(to_drop2)):
  df_crea = df_crea[df_crea.power != to_drop2[k]]

In [ ]:
to_drop2 = ['*','1+*','+1','3.5','?','1.5','0',-1]
for k in range(len(to_drop2)):
  df_crea = df_crea[df_crea.toughness != to_drop2[k]]

In [ ]:
df_crea['power'] = df_crea['power'].astype(float)
df_crea['toughness'] =df_crea['toughness'].astype(float)

In [ ]:
df_crea.corr()

<ipython-input-260-64606f683f3b>:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



,manaValue,power,toughness,price,raridade,classified_text
manaValue,1.000000,0.740712,0.715950,0.041380,0.098093,0.082359
power,0.740712,1.000000,0.704106,0.021910,0.056545,0.069604
toughness,0.715950,0.704106,1.000000,0.025306,0.068864,0.084198
price,0.041380,0.021910,0.025306,1.000000,0.008875,0.718719
raridade,0.098093,0.056545,0.068864,0.008875,1.000000,0.001990
classified_text,0.082359,0.069604,0.084198,0.718719,0.001990,1.000000


# Criando o modelo de classificação de textos

In [ ]:
data_texts = df_crea['text'].astype(str).to_list()
data_labels = df_crea['classified_text'].to_list()
train_texts, val_texts, train_labels, val_labels = train_test_split(data_texts, data_labels, test_size = 0.8, random_state = 0 )

In [ ]:
train_texts

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [ ]:
train_encodings = tokenizer(train_texts, truncation = True, padding = True  )

val_encodings = tokenizer(val_texts, truncation = True, padding = True )

In [ ]:
train_encodings

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
     dict(train_encodings),
     train_labels
 ))


val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
     val_labels
 ))

In [ ]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=5)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [ ]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments


training_args = TFTrainingArguments(
    output_dir='/content/drive/Shareddrives/Ufla/Pattern_Recognition/results',
    num_train_epochs=20,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=50000,
    weight_decay=1e-5,
    logging_dir='/content/drive/Shareddrives/Ufla/Pattern_Recognition/logs',
    eval_steps=1000
)

with training_args.strategy.scope():
    trainer_model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels = 20 )


trainer = TFTrainer(
    model=trainer_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

{'eval_loss': nan}

In [ ]:
save_directory = "/content/drive/Shareddrives/Ufla/Pattern_Recognition/saved_models"

model.save_pretrained(save_directory)

tokenizer.save_pretrained(save_directory)

('/content/drive/Shareddrives/Ufla/Pattern_Recognition/saved_models/tokenizer_config.json',
 '/content/drive/Shareddrives/Ufla/Pattern_Recognition/saved_models/special_tokens_map.json',
 '/content/drive/Shareddrives/Ufla/Pattern_Recognition/saved_models/vocab.txt',
 '/content/drive/Shareddrives/Ufla/Pattern_Recognition/saved_models/added_tokens.json')

In [ ]:
tokenizer_fine_tuned = DistilBertTokenizer.from_pretrained(save_directory)

model_fine_tuned = TFDistilBertForSequenceClassification.from_pretrained(save_directory)

Some layers from the model checkpoint at /content/drive/Shareddrives/Ufla/Pattern_Recognition/saved_models were not used when initializing TFDistilBertForSequenceClassification: ['dropout_99']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /content/drive/Shareddrives/Ufla/Pattern_Recognition/saved_models and are newly initialized: ['dropout_139']
You should probably TRAIN this model on a down-stream task to be able to use it for pr

# Clusterização


In [ ]:
noatributes = ['originalType','text','price','rarity','pred_price']
features = df_crea.drop(columns= [col for col in df_crea if col in noatributes])
target = df_crea['price']

In [ ]:
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=.8, random_state=None) # hold out 20% of the data for final testing

(2122, 5)

In [ ]:
label_encoder = LabelEncoder()

true_labels = label_encoder.fit_transform(target)

In [ ]:
label_encoder.classes_

n_clusters = len(label_encoder.classes_)

print(n_clusters)

In [ ]:
preprocessor = Pipeline(
    [
        ("scaler", MinMaxScaler()),
        ("pca", PCA(n_components=5, random_state=42)),
    ]
)

In [ ]:
clusterer = Pipeline(
    [
        (
            "kmeans",
            KMeans(
                n_clusters=n_clusters,
                init="k-means++",
                n_init=50,
                max_iter=500,
                random_state=42,
            ),
         ),
    ]
 )

In [ ]:
pipe = Pipeline(
     [
         ("preprocessor", preprocessor),
         ("clusterer", clusterer)
     ]
   )

In [ ]:
pipe.fit(features)

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('scaler', MinMaxScaler()),
                                 ('pca',
                                  PCA(n_components=5, random_state=42))])),
                ('clusterer',
                 Pipeline(steps=[('kmeans',
                                  KMeans(max_iter=500, n_clusters=539,
                                         n_init=50, random_state=42))]))])

In [ ]:
preprocessed_data = pipe["preprocessor"].transform(features)

predicted_labels = pipe["clusterer"]["kmeans"].labels_

silhouette_score(preprocessed_data, predicted_labels)

0.7258360548512859

In [ ]:
from sklearn.metrics.cluster import adjusted_rand_score
adjusted_rand_score(true_labels, predicted_labels)

0.013069682411475759

In [ ]:
dfcluster = pd.DataFrame(
    pipe["preprocessor"].transform(features),
    columns=features.columns,
)

dfcluster["predicted_cluster"] = pipe["clusterer"]["kmeans"].labels_
dfcluster["true_label"] = label_encoder.inverse_transform(true_labels)

plt.style.use("fivethirtyeight")
plt.figure(figsize=(8, 8))

scat = sns.scatterplot(
    data=dfcluster[['classified_text','predicted_cluster',"true_label"]],
    x = "classified_text",
    y = "predicted_cluster",
    s=50,
    hue="predicted_cluster",
    style="true_label",
    palette="Set2",
)

scat.set_title(
     "Clustering results Magic/price"
)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)

plt.show()

In [ ]:
dfcluster

,manaValue,power,toughness,raridade,classified_text,predicted_cluster,true_label
0,0.564800,0.180737,-0.045401,-0.075880,-0.063937,148,0.04
1,0.558836,0.037315,-0.031574,-0.041155,-0.029975,27,0.04
2,-0.446936,-0.077428,-0.017868,0.160701,-0.071230,157,0.04
3,-0.444533,-0.030126,-0.024047,-0.045148,-0.038166,33,0.04
4,0.554679,-0.069117,-0.022137,-0.041493,-0.031888,53,0.03
...,...,...,...,...,...,...,...
10607,-0.444534,-0.030216,-0.024973,-0.045143,-0.038175,33,0.03
10608,0.056879,-0.012677,-0.027944,-0.078380,-0.070906,170,0.02
10609,-0.440377,0.076217,-0.034410,-0.044805,-0.036263,8,0.04
10610,-0.450497,-0.173638,-0.011145,-0.010418,-0.004213,0,0.03


In [ ]:
dfcluster['true_label'].loc[dfcluster['predicted_cluster'] == int(pipe.predict(test_input))]

369     0.02
969     0.11
2162    0.95
2494    0.36
2726    0.26
3947    0.02
4449    0.73
6604    0.10
6672    1.21
6990    0.47
7626    0.06
7648    0.02
8065    0.42
8150    0.02
8646    0.02
9791    0.02
Name: true_label, dtype: float64

# Previsão

In [ ]:
test_text = "Flying, vigilance At the beginning of your upkeep, investigate once for each opponent who has more cards in hand than you. "

#test_text

In [ ]:
predict_input = tokenizer_fine_tuned.encode(
    test_text,
    truncation = True,
    padding = True,
    return_tensors = 'tf'
)

output = model_fine_tuned(predict_input)[0]

prediction_value = tf.argmax(output, axis = 1).numpy()[0]

prediction_value

0

In [ ]:
tokenizer_fine_tuned_pt = DistilBertTokenizer.from_pretrained(save_directory)


model_fine_tuned_pt = DistilBertForSequenceClassification.from_pretrained(save_directory, from_tf = True )

All TF 2.0 model weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DistilBertForSequenceClassification for predictions without further training.


In [ ]:
predict_input_pt = tokenizer_fine_tuned_pt.encode(test_text, truncation = True, padding = True, return_tensors = 'pt' )

ouput_pt = model_fine_tuned_pt(predict_input_pt)

prediction_value_pt = torch.argmax(ouput_pt[0], dim = 1 ).item()

prediction_value_pt

0

In [ ]:
# alterar valores conforme atributos da carta a ser testada e o valor de saída de classificação do texto
valores = [[3, 2, 4, 2, 0]]

test_input = pd.DataFrame(valores, columns=['manaValue', 'power', 'toughness', 'raridade', 'classified_text'])

In [ ]:
dfcluster['true_label'].loc[dfcluster['predicted_cluster'] == int(pipe.predict(test_input))].mean()

0.299375

# Deploy versão 2.0


In [ ]:
# dfs = {'creatures': df_crea , 'artefatos' : df_art }
# for key, df in dfs.items():
#   data_texts = df['text'].astype(str).to_list()
#   data_labels = df['classified_text'].to_list()
#   train_texts, val_texts, train_labels, val_labels = train_test_split(data_texts, data_labels, test_size = 0.8, random_state = 0 )

#   train_encodings = tokenizer(train_texts, truncation = True, padding = True  )
#   val_encodings = tokenizer(val_texts, truncation = True, padding = True )

#   train_dataset = tf.data.Dataset.from_tensor_slices((
#     dict(train_encodings),
#     train_labels
#     ))


#   val_dataset = tf.data.Dataset.from_tensor_slices((
#     dict(val_encodings),
#     val_labels
#     ))
#   training_args = TFTrainingArguments(
#     output_dir=f'/content/drive/Shareddrives/Ufla/Pattern_Recognition/results/{key}',
#     num_train_epochs=10,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=64,
#     warmup_steps=10000,
#     weight_decay=1e-5,
#     logging_dir=f'/content/drive/Shareddrives/Ufla/Pattern_Recognition/logs/{key}',
#     eval_steps=100
#     )

#   with training_args.strategy.scope():
#       trainer_model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels = 20 )


#   trainer = TFTrainer(
#       model=trainer_model,
#       args=training_args,
#       train_dataset=train_dataset,
#       eval_dataset=val_dataset,
#   )

#   trainer.train()
#   trainer.evaluate()